In [1]:
import pandas as pd
import numpy as np

In [2]:
'''
2015 FLIGHT DATA
Download data from source to local drive
https://www.kaggle.com/usdot/flight-delays
'''

flights_df = pd.read_csv("flights.csv")
airlines_df = pd.read_csv("airlines.csv")
airports_df = pd.read_csv("airports.csv")

C:\Users\Andrew Chin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
'Merge data: Airport & Airline files are lookup definitions

combined_df = pd.merge(flights_df, airlines_df, left_on='AIRLINE', right_on='IATA_CODE')
combined_df = pd.merge(combined_df, airports_df, left_on='ORIGIN_AIRPORT', right_on='IATA_CODE')

In [4]:
'combined_df = primary DF
'correlated_df = DF to test and drop correlated fields

combined_df = combined_df[['MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE_x', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'LATITUDE', 'LONGITUDE', 'CITY', 'STATE', 'COUNTRY', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'SCHEDULED_TIME', 'SCHEDULED_ARRIVAL', 'DISTANCE', 'ARRIVAL_DELAY', 'DEPARTURE_DELAY']]
correlated_df = combined_df[['MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE_x', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'LATITUDE', 'LONGITUDE', 'CITY', 'STATE', 'COUNTRY', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'SCHEDULED_TIME', 'SCHEDULED_ARRIVAL', 'DISTANCE', 'ARRIVAL_DELAY']]

In [5]:
'Test correlation matrix

corr_matrix = correlated_df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]

In [6]:
to_drop

['DEPARTURE_TIME', 'DISTANCE']

In [7]:
'DEPARTURE_TIME & DISTANCE not needed

combined_df = combined_df.drop('DEPARTURE_TIME', axis=1)
combined_df = combined_df.drop('DISTANCE', axis=1)
combined_df.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,ORIGIN_AIRPORT,LATITUDE,LONGITUDE,CITY,STATE,COUNTRY,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,DEPARTURE_DELAY
0,1,1,4,AS,98,ANC,61.17432,-149.99619,Anchorage,AK,USA,SEA,5,205.0,430,-22.0,-11.0
1,1,1,4,AS,108,ANC,61.17432,-149.99619,Anchorage,AK,USA,SEA,45,204.0,509,-14.0,-4.0
2,1,1,4,AS,122,ANC,61.17432,-149.99619,Anchorage,AK,USA,PDX,50,215.0,525,-18.0,-4.0
3,1,1,4,AS,136,ANC,61.17432,-149.99619,Anchorage,AK,USA,SEA,135,205.0,600,NaN,NaN
4,1,1,4,AS,134,ANC,61.17432,-149.99619,Anchorage,AK,USA,SEA,155,218.0,633,-35.0,-15.0


In [8]:
'Translate departure delay as independed variable to bit: 0 = early/on-time, 1 = delayed

combined_df['DEPARTURE_DELAY'] = np.where(combined_df['DEPARTURE_DELAY'] > 0, 1, 0).astype('int')

In [9]:
combined_df.head(10)

,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,ORIGIN_AIRPORT,LATITUDE,LONGITUDE,CITY,STATE,COUNTRY,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,SCHEDULED_TIME,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,DEPARTURE_DELAY
0,1,1,4,AS,98,ANC,61.17432,-149.99619,Anchorage,AK,USA,SEA,5,205.0,430,-22.0,0
1,1,1,4,AS,108,ANC,61.17432,-149.99619,Anchorage,AK,USA,SEA,45,204.0,509,-14.0,0
2,1,1,4,AS,122,ANC,61.17432,-149.99619,Anchorage,AK,USA,PDX,50,215.0,525,-18.0,0
3,1,1,4,AS,136,ANC,61.17432,-149.99619,Anchorage,AK,USA,SEA,135,205.0,600,NaN,0
4,1,1,4,AS,134,ANC,61.17432,-149.99619,Anchorage,AK,USA,SEA,155,218.0,633,-35.0,0
5,1,1,4,AS,144,ANC,61.17432,-149.99619,Anchorage,AK,USA,PDX,200,210.0,630,-11.0,0
6,1,1,4,AS,114,ANC,61.17432,-149.99619,Anchorage,AK,USA,SEA,220,200.0,640,-12.0,0
7,1,1,4,AS,730,ANC,61.17432,-149.99619,Anchorage,AK,USA,SEA,505,205.0,930,-14.0,0
8,1,1,4,AS,157,ANC,61.17432,-149.99619,Anchorage,AK,USA,OTZ,630,102.0,812,-10.0,0
9,1,1,4,AS,112,ANC,61.17432,-149.99619,Anchorage,AK,USA,SEA,700,210.0,1130,-15.0,0


In [10]:
'Maximize accuracy of model by dropping all missing values

combined_df = combined_df.dropna()

In [11]:
'''Reduce dataset to 25% due to memory errors'''

combined_df = combined_df.sample(frac=0.25, replace=True, random_state=1)

In [12]:
'Get Dummies converts fields with text values into bit-value columns

combined_df = pd.get_dummies(combined_df)

In [13]:
'Set X & y datasets to pass into model

X = combined_df.drop('DEPARTURE_DELAY', axis=1)
y = combined_df['DEPARTURE_DELAY']

In [14]:
'''
Split data into train 75% and test 25%
Stratify = y splits data with even distribution of 0/1 independent variables in each set
'''

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [15]:
'Prepare logistic model

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
'Fit X & y datasets into model

model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
'Not bad: 81% accuracy

print(f"Training Data Score: {model.score(X_train, y_train)}")
print(f"Testing Data Score: {model.score(X_test, y_test)}")

Training Data Score: 0.810236116822673
Testing Data Score: 0.8108467187696113


In [18]:
'Check the precision and recall results from the model

predictions = model.predict(X_test)
overallresults = pd.DataFrame({"Prediction": predictions, "Actual": y_test})

In [19]:
'Accuracy of 92% achived for early/on-time flights and 64% for delayed flights

from sklearn.metrics import classification_report
predict =  model.predict(X_test)
print(classification_report(y_test,predict))

             precision    recall  f1-score   support

          0       0.81      0.92      0.86    203528
          1       0.82      0.64      0.72    123133

avg / total       0.81      0.81      0.80    326661



In [21]:
'''Save model into local file directory using Pickle'''

import pickle

filename = 'predictingflightdelays.sav'
pickle.dump(model, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))

In [22]:
'Test loading data into the saved model

result = loaded_model.score(X_test, y_test)
print(result)

0.8108467187696113
